In [1]:
"|IMPORT PACKAGES|"
import numpy              as np
import pandas             as pd
import datetime
from   math               import pi
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label, BoxAnnotation, DatetimeTickFormatter, Text, Span
from   bokeh.models.tools import HoverTool
from   bokeh.models       import Arrow, NormalHead, OpenHead, VeeHead
from   bokeh.transform    import dodge
from   datetime           import datetime as dt

In [37]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Internacional.xlsx?raw=true'
#file = r'Resumen%20Estadistico%20-%20Argentina.xlsx'
IO   = path + file

sheet = 'DATA'

data = pd.read_excel(IO, sheet_name = sheet, usecols="A:E,S", skiprows=2, engine='openpyxl') # Be patient...

data = data[data.YEAR == 2018].reset_index()
data = data.rename(columns={"LABOR_SHARE":"LABOR"})
data = data.drop(['index'], axis = 1)
data["LABOR"] = data["LABOR"]*100

In [45]:
"|CHECK DATA|"
data = data.dropna()
data

,YEAR,ISO CODE,COUNTRY,REGION,EFW,LABOR
1,2018,AGO,Angola,Africa,4.749723,33.114234
2,2018,BEN,Benin,Africa,6.199911,61.715502
3,2018,BWA,Botswana,Africa,7.603481,27.796099
5,2018,BFA,Burkina Faso,Africa,5.852735,46.092802
6,2018,BDI,Burundi,Africa,5.943080,60.621822
...,...,...,...,...,...,...
215,2018,CAN,Canada,North America,8.170000,65.488040
218,2018,USA,United States,North America,8.222738,59.432596
220,2018,AUS,Australia,Oceania,8.230509,57.164514
224,2018,FJI,Fiji,Oceania,6.502444,48.862648


In [70]:
"|BUILD TRENDLINES|"
"Africa"
x_africa = data.loc[data["REGION"]=="Europe", "EFW"]
y_africa = data.loc[data["REGION"]=="Europe", "LABOR"]

africa       = np.polyfit(x_africa, y_africa, 1)
trend_africa = africa[1] + africa[0]*x_africa

"Asia"
x_asia = data.loc[data["REGION"]=="Asia", "EFW"]
y_asia = data.loc[data["REGION"]=="Asia", "LABOR"]

asia       = np.polyfit(x_asia, y_asia, 1)
trend_asia = asia[1] + asia[0]*x_asia

"Europe"
x_europe = data.loc[data["REGION"]=="Europe", "EFW"]
y_europe = data.loc[data["REGION"]=="Europe", "LABOR"]

europe       = np.polyfit(x_europe, y_europe, 1)
trend_europe = europe[1] + europe[0]*x_europe

"Latin America"
x_latam = data.loc[data["REGION"]=="Latin America", "EFW"]
y_latam = data.loc[data["REGION"]=="Latin America", "LABOR"]

latam       = np.polyfit(x_latam, y_latam, 1)
trend_latam = latam[1] + latam[0]*x_latam

"North America"
x_noram = data.loc[data["REGION"]=="North America", "EFW"]
y_noram = data.loc[data["REGION"]=="North America", "LABOR"]

noram       = np.polyfit(x_noram, y_noram, 1)
trend_noram = noram[1] + noram[0]*x_noram

"Oceania"
x_oceania = data.loc[data["REGION"]=="Oceania", "EFW"]
y_oceania = data.loc[data["REGION"]=="Oceania", "LABOR"]

oceania       = np.polyfit(x_oceania, y_oceania, 1)
trend_oceania = oceania[1] + oceania[0]*x_oceania

In [87]:
"|BUILD PLOT|"
colormap = {'Africa'       :'black',
            'Latin America':'teal'  ,
            'North America':'red'   ,
            'Asia'         :'olive' ,
            'Europe'       :'purple',
            'Oceania'      :'blue'}  

colors = [colormap[i] for i in data['REGION']]

data['COLORS'] = colors

x = data['EFW']
y = data['LABOR']

cds = ColumnDataSource(data)

#BUILD FIGURE 1
p = figure(title = "EL HUB ECONÓMICO | ÍNDICE DE LIBERTAD ECONÓMICA E INGRESO FACTOR TRABAJO (%PBI)",
          # x_range      = (2,10),
           x_axis_label = "(-) Índice de libertad económica (+)",
           plot_height  = 400,
           plot_width   = 700)

p1 = p.circle('EFW', 'LABOR', fill_color='COLORS', fill_alpha=0.50, line_color='COLORS', legend_group='REGION', line_alpha=0.50, size=8, source=cds)
p2 = p.circle('EFW', 'COUNTRY', fill_alpha=0, line_alpha=0, size=1, source=cds)
p3 = p.circle('EFW', 'REGION' , fill_alpha=0, line_alpha=0, size=1, source=cds)

p4 = p.line(x_africa , trend_africa , alpha=0.5, color='black')
p5 = p.line(x_asia   , trend_asia   , alpha=0.5, color='olive')
p6 = p.line(x_europe , trend_europe , alpha=0.5, color='purple')
p7 = p.line(x_latam  , trend_latam  , alpha=0.5, color='teal')
#p8 = p.line(x_noram  , trend_noram  , alpha=0.5, color='red')
p9 = p.line(x_oceania, trend_oceania, alpha=0.5, color='blue')


p.add_tools(HoverTool(renderers=[p1], tooltips = [("País","@COUNTRY"),("Regions","@REGION"),("Índce", "@EFW{0.0}"),("Trabajo (%)", "@LABOR{$0,0}")]))


year = Label(x=3.5, y=25, text="2018", text_font_size='20px', text_alpha=0.75, text_align="center", text_baseline="middle")
p.add_layout(year)

p.legend.location     = "top_left"
p.legend.orientation  = "horizontal"
output_notebook()
show(p)

Loading BokehJS ...

In [ ]:
"|EXPORT .PNG FILE|"

export_png(p, filename="efw_labor_share.png")

In [ ]:
"|# CREATE HTML FILE|"

output_file(filename="efw_labor_share.html", title="EFW y TRABAJO")
save(p)

In [180]:
"|CREATE JSON FILE|"
import json
import bokeh.embed

from bokeh.embed import json_item

j = json.dumps(json_item(p, "efw_labor_share"))
with open("07.10_efw_labor_share.json", "w") as fp:
    json.dump(j, fp)